
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# The next 8 lines added to be compatible with version changes (2022/10/13).
# change cuda to 11.1
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.1/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.1/lib64:{ld}"
# change pytorch to 1.9.0 compiled with cuda 11.1
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html


# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

/content
'=2.0.1'   sample_data
Sat Mar  4 05:07:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                        

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
from torchsummary import summary
from glob import glob
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import GenericMask
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settings in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

True

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

Mounted at /content/drive


In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = f'{BASE_DIR}/output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection
See this tutorial: https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5#scrollTo=PIbAM2pv-urF

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''
def get_detection_data(set_name, val_size = 30):
  dataset = []

  ### test set
  if set_name == 'test':
    
    for f in tqdm(glob(f'{BASE_DIR}/data/test/*')):
      record = {}
      record['file_name'] = f
      record['height'], record['width'] = cv2.imread(record['file_name']).shape[:2]
      record['annotations'] = []
      dataset.append(record)

  ### training/validation set
  else:

    np.random.seed(1234)
    with open(f'{BASE_DIR}/data/train.json') as f: imgs_anns = json.load(f)

    for i, j in enumerate(tqdm(imgs_anns)):
      record = {}

      record['file_name'] = os.path.join(f'{BASE_DIR}/data/train', j['file_name'])
      annos = j['segmentation']

      px = [annos[0][x] for x in range(0, len(annos[0]), 2)]
      py = [annos[0][y] for y in range(1, len(annos[0]), 2)]
      poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
      poly = [p for x in poly for p in x]
      record['annotations'] = [{'bbox': [np.min(px), np.min(py), np.max(px), np.max(py)],
                                'bbox_mode': BoxMode.XYXY_ABS,
                                'segmentation': [poly],
                                'category_id': 0,}]

      k = next((ind for (ind, d) in enumerate(dataset) if d["file_name"] == record['file_name']), None)

      if k != None: 
        dataset[k]['annotations'].append(record['annotations'][0])
        continue

      record['height'], record['width'] = cv2.imread(record['file_name']).shape[:2]
      record['image_id'] = i
      dataset.append(record)

    validation_set = np.random.choice(dataset, val_size, replace = False)                                     # validation set size: 30 (15% of training set)
    if set_name == 'val': return list(validation_set)
    dataset = np.delete(dataset, np.isin(dataset, validation_set))

  return list(dataset)

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consider "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by splitting the training data
# TODO: approx 5 lines
'''
for d in ['train', 'test', 'val']:
    DatasetCatalog.register('data_detection_' + d, lambda d = d: get_detection_data(d))
    MetadataCatalog.get('data_detection_' + d).set(thing_classes = ['plane'])
plane_metadata = MetadataCatalog.get('data_detection_train')

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset_dicts = get_detection_data('train')
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d['file_name'])
    visualizer = Visualizer(img[:, :, ::-1], metadata = plane_metadata, scale = 0.25)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00075
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []
cfg.SOLVER.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts_test = get_detection_data('test')
for d in random.sample(dataset_dicts_test, 3):    
    im = cv2.imread(d['file_name'])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata = plane_metadata, 
                   scale = 0.25, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    out = v.draw_instance_predictions(outputs['instances'].to('cpu'))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''

evaluator = COCOEvaluator('data_detection_val', output_dir = './output')
val_loader = build_detection_test_loader(cfg, 'data_detection_val')
print(inference_on_dataset(trainer.model, val_loader, evaluator))

## Part 2: Semantic Segmentation

In [ ]:
plane_data = DatasetCatalog.get('data_detection_train')
plane_data_valid = DatasetCatalog.get('data_detection_val')
plane_data_test = DatasetCatalog.get('data_detection_test')

### Data Loader

In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
def get_instance_sample(data, img = None):

  ### used cv2.resize() instead of np.resize() because want image to be 
  ### 128x128 in resolution, not resized into an array of size 128x128
  ### reference: https://stackoverflow.com/questions/68729468/difference-between-opencv-and-numpy-resize-functions
  cropped_planes = {'obj_img': [], 'obj_mask':[]}
  img = cv2.imread(data['file_name'])
  for i in range(len(data['annotations'])):
      annos = data['annotations'][i]
      bbox = annos['bbox']
      obj_img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
      obj_img = cv2.resize(obj_img, (128, 128), interpolation = cv2.INTER_AREA) 
      obj_mask = GenericMask(annos['segmentation'], data['height'], data['width']).mask
      obj_mask = obj_mask[bbox[1]:bbox[3], bbox[0]:bbox[2]] * 255
      obj_mask = cv2.resize(obj_mask, (128, 128), interpolation = cv2.INTER_AREA)
      cropped_planes['obj_img'].append(obj_img)
      cropped_planes['obj_mask'].append(obj_mask)
  return cropped_planes

cropped_planes = {'obj_img':[], 'obj_mask':[]}
for i in tqdm(plane_data_valid):
    res = get_instance_sample(i)
    cropped_planes['obj_img'] += [res['obj_img']]
    cropped_planes['obj_mask'] += [res['obj_mask']]

  0%|          | 0/168 [00:00<?, ?it/s]

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          transforms.ToTensor(), # Converting the image to tensor and change the image format (Channels-Last => Channels-First)
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)                # convert PIL image to tensor
        mask = self.transforms(mask)              # convert PIL image to tensor
    img = img.clone().detach().float()
    mask = mask.clone().detach().float()
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]

    ### runtime is faster if 'img' and 'mask' are called from memory, as
    ### opposed to calling get_instance_sample() every time __getitem__ is called
    img = cropped_planes['obj_img'][idx[0]][idx[1]]
    mask = cropped_planes['obj_mask'][idx[0]][idx[1]]
    img, mask = self.numpy_to_tensor(img, mask)
    return img, mask

def get_plane_dataset(set_name = 'train', batch_size = 2):
    my_data_list = plane_data if set_name == 'train' else DatasetCatalog.get('data_detection_{}'.format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=2,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2, 2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    
    # Encoder
    self.conv_in = conv(3, 64)
    self.down1 = down(64, 64)
    self.down2 = down(64, 128)
    self.down3 = down(128, 128)
    self.down4 = down(128, 256)
    self.down5 = down(256, 256)
    self.down6 = down(256, 512)
    self.down7 = down(512, 512)
    
    # Decoder
    self.up8 = up(512, 512)
    self.up7 = up(512, 256)
    self.up6 = up(256, 256)
    self.up5 = up(256, 128)
    self.up4 = up(128, 128)
    self.up3 = up(128, 64)
    self.up2 = up(64, 64)
    self.up1 = conv(64, 3)
    self.conv_out = conv(3, 1, False)
    
    self.bn1 = nn.BatchNorm2d(64)
    self.bn2 = nn.BatchNorm2d(128)
    self.bn3 = nn.BatchNorm2d(256)
    self.bn4 = nn.BatchNorm2d(512)

  def forward(self, input):
    x1 = self.conv_in(input)
    x2 = self.down1(x1)
    x3 = self.down2(x2)
    x4 = self.down3(x3)
    x5 = self.down4(x4)
    x6 = self.down5(x5)
    x7 = self.down6(x6)
    x8 = self.down7(x7)
    
    x8 = F.relu(self.bn4(self.up8(x8)))
    x7 = F.relu(self.bn3(self.up7(x7)))
    x6 = F.relu(self.bn3(self.up6(x6)))
    x5 = F.relu(self.bn2(self.up5(x5)))
    x4 = F.relu(self.bn2(self.up4(x4)))
    x3 = F.relu(self.bn1(self.up3(x3)))
    x2 = F.relu(self.bn1(self.up2(x2)))
    x1 = F.relu(self.up1(x1))
    output = self.conv_out(x1)
    
    return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''

# Set the hyperparameters
num_epochs = 25
batch_size = 32
learning_rate = 0.01
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
for epoch in tqdm(range(num_epochs)):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = img.clone().detach().requires_grad_(True).cuda()
    mask = mask.clone().detach().requires_grad_(True).cuda()
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))

In [ ]:
summary(model, (3, 128, 128))

### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''

### reference: https://stackoverflow.com/questions/60746851/sigmoid-function-in-numpy
### reference: https://towardsdatascience.com/metrics-to-evaluate-your-semantic-segmentation-model-6bcb99639aa2
### reference: discussed with colleagues

batch_size = 1
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # changing the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('val', batch_size)

total_iou = 0

for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cpu().detach()
    pred = model(img).cpu().detach()

    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    
    for i in range(img.shape[0]):
      gt = np.array(mask[i])[0]
      gt = np.rint(1/(1 + np.exp(-gt)))
      det = np.array(pred[i])[0]
      det = np.rint(1/(1 + np.exp(-det)))
      union = np.count_nonzero(gt + det)
      if union == 0: continue
      intersection = np.count_nonzero(np.multiply(gt, det))
      total_iou += (intersection/union)

print("\n #images: {}, Mean IoU: {}".format(len(loader), total_iou / len(loader)))

  0%|          | 0/7980 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]


 #images: 812, Mean IoU: 0.5731136525344944


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''

batch_size = 3
loader, _ = get_plane_dataset('train', batch_size)

for (img, mask) in loader:
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cpu().detach()
    pred = model(img).cpu().detach()
    for i in range(batch_size):
      ### convert tensor to PIL image
      res_img = transforms.ToPILImage()(img[i].cpu())
      cv2_imshow(np.array(res_img))
      res_pred = np.rint(1/(1 + np.exp(-np.array(pred[i])[0]))) * 255
      cv2_imshow(res_pred)
      res_mask = np.array(mask[i])[0] * 255
      cv2_imshow(res_mask)
    break

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this function) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

def get_prediction_mask(data, set_name):
  img = cv2.imread(data['file_name'])
  img_original = np.zeros([data['height'], data['width']])                                                  # np.array of equivalent dimensions as original image

  """ 
  algorithm: obtain predicted masks
    - for every bounding box (a cropped image of a plane)
      - resize PIL image to 128 x 128
      - convert PIL image to a tensor
      - adjust tensor shape into correct dimensions
        - check previous section to determine correct size, see size of
          'img', 'mask', and/or 'pred'
      - using our model, get a prediction on the cropped plane (tensor)
      - convert tensor to a np.array (PIL image)
      - resize PIL image to original size
      - set pixel values on original size of plane 
      - deal with overlapping bbox
  """

  instances = predictor(img)['instances'] if set_name == 'test' else data['annotations']                    # input data: plane identification
  if len(instances) == 0: pred_mask = transforms.ToTensor()(img_original).cuda()

  ### for every plane identified...
  for i in range(len(instances)):

    ### set bounding boxes accordingly (train/valid or test dataset)
    if set_name == 'test':
      bbox = next(iter(instances[i]._fields['pred_boxes']))
      bbox = bbox.cpu().detach().numpy()
      bbox = np.rint(bbox).astype(int)
    else: bbox = instances[i]['bbox']

    ### get predicted image of planes from every bounding box
    cropped_plane = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]                                                   # crop plane from image (list), H x W x C --> H x W x 3
    cropped_plane = cv2.resize(cropped_plane, (128, 128), interpolation = cv2.INTER_AREA)                   # resize crop to 128x128, H x W x C --> 128 x 128 x 3
    cropped_plane = transforms.ToTensor()(cropped_plane).cuda()                                             # PIL image to tensor, C x H x W --> 3 x 128 x 128
    cropped_plane = torch.unsqueeze(cropped_plane, 0)                                                       # add a dimension in the beginning of the tensor to match correct size
    pred = model(cropped_plane)                                                                             # predicted image (tensor)
    pred = pred.cpu().detach().numpy()[0]                                                                   # convert tensor to np.array
    pred = cv2.resize(pred[0], (bbox[2] - bbox[0], bbox[3] - bbox[1]), interpolation = cv2.INTER_AREA)      # resize 128 x 128 to original size
    pred = np.rint(1/(1 + np.exp(-pred))) * (255 - i*5)                                                     # predicted image visualization
  
    ### deal with overlapping pixels --> consider when two bbox overlap one another, keep largest pixel value
    bbox_black = img_original[bbox[1]:bbox[3], bbox[0]:bbox[2]]                                             # bbox pixel values
    bbox_updated = np.maximum(bbox_black, pred)                                                             # per pixel: max(0, predicted value); predicted value either 0 or plane
    img_original[bbox[1]:bbox[3], bbox[0]:bbox[2]] = bbox_updated                                           # update original image with bbox pixels
    
    pred_mask = transforms.ToTensor()(img_original).cuda()

  """
  algorithm: obtain ground truth masks
    - gt_mask = np.zeros([data['height'], data['width']])
      - note that gt_mask will be all 0s for the test set
    - for every bounding box
      - get mask
      - update gt_mask
  """

  gt_mask = np.zeros([data['height'], data['width']])                                                       # np.array of equivalent dimensions as original image
  for i in range(len(data['annotations'])):
    bbox = data['annotations'][i]['bbox']
    mask = GenericMask(data['annotations'][i]['segmentation'], data['height'], data['width']).mask
    bbox_updated = np.maximum(gt_mask[bbox[1]:bbox[3], bbox[0]:bbox[2]], mask[bbox[1]:bbox[3], bbox[0]:bbox[2]] * (255 - (i * 5)))
    gt_mask[bbox[1]:bbox[3], bbox[0]:bbox[2]] = bbox_updated
  gt_mask = transforms.ToTensor()(gt_mask).cuda()

  return img, gt_mask, pred_mask

### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
for i in np.random.choice(len(plane_data), 3, replace = False):
  img, true_mask, pred_mask = get_prediction_mask(plane_data[i], 'train')
  pred_mask = pred_mask.cpu().numpy()
  true_mask = true_mask.cpu().numpy()
  cv2_imshow(cv2.resize(img, (img.shape[1]//5, img.shape[0]//5)))
  cv2_imshow(cv2.resize(true_mask[0], (img.shape[1]//5, img.shape[0]//5)))
  cv2_imshow(cv2.resize(pred_mask[0], (img.shape[1]//5, img.shape[0]//5)))

In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''
# plane_data = DatasetCatalog.get('data_detection_train')
plane_data_valid = DatasetCatalog.get('data_detection_val')
plane_data_test = DatasetCatalog.get('data_detection_test')

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''

for i in tqdm(range(len(plane_data)), position=0, leave=True):
  sample = plane_data[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, 'train')
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

### validation set
for i in tqdm(range(len(plane_data_valid)), position=0, leave=True):
  sample = plane_data_valid[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, 'train')
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

'''
# Writing the predictions of the test set
'''

for i in tqdm(range(len(plane_data_test)), position=0, leave=True):
  sample = plane_data_test[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask = get_prediction_mask(sample, 'test')
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()

## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
dataset_dicts = get_detection_data('train')
for d in ['train', 'test', 'val']:
    DatasetCatalog.register('data_detection_' + d, lambda d = d: get_detection_data(d))
    MetadataCatalog.get('data_detection_' + d).set(thing_classes = ['plane'])
plane_metadata = MetadataCatalog.get('data_detection_train')

### Network

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00075
cfg.SOLVER.MAX_ITER = 500
cfg.SOLVER.STEPS = []
cfg.SOLVER.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

### Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume = False)
trainer.train()

### Evaluation and Visualization

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
predictor = DefaultPredictor(cfg)

In [ ]:
dataset_dicts_test = get_detection_data('test')
#for d in random.sample(dataset_dicts_test, 3):    
for d in (dataset_dicts_test[20], dataset_dicts_test[30], dataset_dicts_test[70]):
    im = cv2.imread(d['file_name'])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata = plane_metadata, 
                   scale = 0.25, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    out = v.draw_instance_predictions(outputs['instances'].to('cpu'))
    cv2_imshow(out.get_image()[:, :, ::-1])